In [2]:
import preprocessing1st as pre1
import preprocessing_nth as prenth
import modeling_score as mdsc

# 1st preprocessing
pre = pre1.Preprocessing1st("../MACH_data/data.csv")
xy = pre.preprocessing_model()


# Nth preprocessing
nth = prenth.PreprocessingNth()

# feature_selection
fs = nth.feature_selection(*xy)

# feature_addition
fa = nth.feature_addition(*fs)

In [8]:
X_train, X_test, y_train, y_test = fa

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((46661, 229), (11666, 229), (46661, 1), (11666,))

In [46]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier


In [60]:
# gridsearch 클래스
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

class Grid_search:    
    def __init__(self, *xy_train_test, **lgb_param):
        lgb = LGBMClassifier()
        
        lgb_param = {
            'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5, 10]
        }
        self.lgb_param = lgb_param
        self.datas = []
        self.grid = GridSearchCV(estimator=lgb, param_grid=self.lgb_param, cv=5)
        self.X_train, self.X_test, self.y_train, self.y_test = xy_train_test
        
    def gridsearch(self):
        self.grid.fit(self.X_train, self.y_train)
        self.y_pre_test = self.grid.predict(self.X_test)
        total_score = self.get_score(self.y_pre_test)
        
        return total_score
        # 분류모델 평가지표 계산함수 (AUC, ACC를 우선순위로 사용함)
    def get_score(self, pred):
        acc = accuracy_score(self.y_test, pred)
        pre = precision_score(self.y_test, pred)
        rec = recall_score(self.y_test, pred)
        f1 = f1_score(self.y_test, pred)
        auc = roc_auc_score(self.y_test, pred)
       
        return acc, auc, pre, rec, f1
    
    def models_score_df(self):
        cols_names = ['accuracy', 'AUC', 'precision', 'recall', 'f1']

        for model in self.models:
            self.datas.append(self.gridsearch)

        df = pd.DataFrame(self.datas, columns=cols_names)
            
        return print(df) 
    
    
    # 평가지표와 confusion matrix 출력 함수
    def print_score(self):
        datas = []
        for model in self.models:
            datas.append(self.fit_model(model))
        
            acc, auc, pre, rec, f1 = datas[0]
            con = confusion_matrix(self.y_test, self.y_pre_test)
            print('='*20)
            print(model)
            print('confusion matrix')
            print(con)
            print('='*20)

            print('Accuracy: {0:.4f}, AUC: {1:.4f}'.format(acc, auc))
            print('Recall: {0:.4f}, f1_score: {1:.4f}, precision: {2:.4f}'.format(rec, f1, pre))
            print('='*20)

In [63]:
a = Grid_search(*fa)

In [64]:
a.grid

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5, 2, 5],
                         'max_depth': [3, 4, 5, 10],
                         'min_child_weight': [1, 5, 10],
                         'subsample': [0.6, 0.8, 1.0]})

In [66]:
a = LGBMClassifier(gridsearch.best_params_)

In [67]:
a.fit(X_train, y_train)

TypeError: Unknown type of parameter:boosting_type, got:dict

In [48]:
lgb_param = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5, 10]
}

In [49]:
gridsearch = GridSearchCV(estimator=lgb, param_grid=lgb_param, cv=5)

In [50]:
gridsearch.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LGBMClassifier(max_depth=2, rabdom_state=13),
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'gamma': [0.5, 1, 1.5, 2, 5],
                         'max_depth': [3, 4, 5, 10],
                         'min_child_weight': [1, 5, 10],
                         'subsample': [0.6, 0.8, 1.0]})

In [51]:
gridsearch.best_score_

0.6550222929858787

In [52]:
gridsearch.best_params_

{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'max_depth': 10,
 'min_child_weight': 10,
 'subsample': 0.6}

In [68]:
gridsearch.best_estimator_

LGBMClassifier(colsample_bytree=0.6, gamma=0.5, max_depth=10,
               min_child_weight=10, rabdom_state=13, subsample=0.6)

In [69]:
import pickle

In [76]:

# Best XGB model that was found based on the metric score you specify
best_model = gridsearch.best_estimator_
# Save model
pickle.dump(gridsearch.best_estimator_, open("gridsearch.pickle", "wb"))

In [77]:
#불러오기
f = open("gridsearch.pickle", "rb")

aaaa = pickle.load(f)

f.close()

In [92]:
pred_1 = aaa.predict(X_test)

In [118]:
%%writefile lgbgrid.py
# gridsearch 클래스
import pickle
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

class GridSearch:    
    def __init__(self, *xy_train_test, **lgb_param):
        lgb = LGBMClassifier()
        
        lgb_param = {
            'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5, 10]
        }
        self.lgb_param = lgb_param
        self.gridsearch = GridSearchCV(estimator=lgb, param_grid=self.lgb_param, cv=5)
        self.X_train, self.X_test, self.y_train, self.y_test = xy_train_test
        
    def model_train(self):
        self.gridsearch.fit(self.X_train, self.y_train)
        
    def model_save(self):
        # Save model
        pickle.dump(self.gridsearch.best_estimator_, open("gridsearch.pickle", "wb"))
        
    def model_load(self):
        f = open("gridsearch.pickle", "rb")
        md = pickle.load(f)
        f.close()
        return md

Writing lgbgrid.py


In [116]:
bb = GridSearch(*fa)

In [117]:
cc = bb.model_train()

KeyboardInterrupt: 

In [104]:
aaa = bb.model_load()

In [91]:
aaa

LGBMClassifier(colsample_bytree=0.6, gamma=0.5, max_depth=10,
               min_child_weight=10, rabdom_state=13, subsample=0.6)

In [58]:
pred = gridsearch.predict(X_test)

In [89]:
acc = accuracy_score(y_test, pred_1)
pre = precision_score(y_test, pred_1)
rec = recall_score(y_test, pred_1)
f1 = f1_score(y_test, pred_1)
auc = roc_auc_score(y_test, pred_1)

acc, auc, pre, rec, f1

(0.6584947711297788,
 0.6546522222303692,
 0.6514911396052442,
 0.7428946936093314,
 0.6941971139085048)

In [59]:
acc = accuracy_score(y_test, pred)
pre = precision_score(y_test, pred)
rec = recall_score(y_test, pred)
f1 = f1_score(y_test, pred)
auc = roc_auc_score(y_test, pred)

acc, auc, pre, rec, f1

(0.6584947711297788,
 0.6546522222303692,
 0.6514911396052442,
 0.7428946936093314,
 0.6941971139085048)

In [112]:
class Modeling2(Modeling):
    def __init__(self, *xy_train_test, unit):
        ada = AdaBoostClassifier()
        gbc = GradientBoostingClassifier()
        xgb = XGBClassifier()
        lgbm = LGBMClassifier()
        grid_lgb= unit
        self.datas = []
        self.models = [ada, gbc, xgb, lgbm, grid_lgb]
        self.model_names = ['Ada', 'GBC', 'XGB', 'LGBM', "grid_lgb"]
        
        self.X_train, self.X_test, self.y_train, self.y_test = xy_train_test

In [105]:
class Modeling2(Modeling):
    def __init__(self, *xy_train_test, unit):
        ada = AdaBoostClassifier()
        gbc = GradientBoostingClassifier()
        xgb = XGBClassifier()
        lgbm = LGBMClassifier()
        unit.model = unit
        grid = unit.model
        self.datas = []
        self.models = [ada, gbc, xgb, lgbm]
        self.model_names = ['Ada', 'GBC', 'XGB', 'LGBM']
        
        self.X_train, self.X_test, self.y_train, self.y_test = xy_train_test

In [113]:
wtf_1 = Modeling2(*fa, unit=aaa)

In [114]:
wtf_1.models_score_df()

          accuracy       AUC  precision    recall        f1
Ada       0.647523  0.643166   0.639615  0.743223  0.687538
GBC       0.652152  0.647497   0.641789  0.754395  0.693551
XGB       0.649151  0.646140   0.648496  0.715295  0.680259
LGBM      0.658409  0.654488   0.650962  0.744538  0.694613
grid_lgb  0.658495  0.654652   0.651491  0.742895  0.694197


In [94]:
wtf = Modeling(*fa)

In [33]:
import numpy as np
import pandas as pd
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, confusion_matrix, classification_report)
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import (train_test_split, cross_val_score, StratifiedKFold ,GridSearchCV)
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

# ada, gbc, xgb, lgbm 모델링 클래스 
class Modeling:    
    def __init__(self, *xy_train_test):
        ada = AdaBoostClassifier()
        gbc = GradientBoostingClassifier()
        xgb = XGBClassifier()
        lgbm = LGBMClassifier()
        self.datas = []
        self.models = [ada, gbc, xgb, lgbm]
        self.model_names = ['Ada', 'GBC', 'XGB', 'LGBM']
        
        self.X_train, self.X_test, self.y_train, self.y_test = xy_train_test
        
        
        # 분류모델 평가지표 계산함수 (AUC, ACC를 우선순위로 사용함)
    def get_score(self, pred):
        acc = accuracy_score(self.y_test, pred)
        pre = precision_score(self.y_test, pred)
        rec = recall_score(self.y_test, pred)
        f1 = f1_score(self.y_test, pred)
        auc = roc_auc_score(self.y_test, pred)
       
        return acc, auc, pre, rec, f1
    
    
    def fit_model(self, model):

        model.fit(self.X_train, self.y_train)
        y_pre_tr = model.predict(self.X_train)
        self.y_pre_test = model.predict(self.X_test)
        total_score = self.get_score(self.y_pre_test)
        
        return total_score

    
    def models_score_df(self):
        cols_names = ['accuracy', 'AUC', 'precision', 'recall', 'f1']

        for model in self.models:
            self.datas.append(self.fit_model(model))

        df = pd.DataFrame(self.datas, columns=cols_names, index=self.model_names)
            
        return print(df) 
    
    
    # 평가지표와 confusion matrix 출력 함수
    def print_score(self):
        datas = []
        for model in self.models:
            datas.append(self.fit_model(model))
        
            acc, auc, pre, rec, f1 = datas[0]
            con = confusion_matrix(self.y_test, self.y_pre_test)
            print('='*20)
            print(model)
            print('confusion matrix')
            print(con)
            print('='*20)

            print('Accuracy: {0:.4f}, AUC: {1:.4f}'.format(acc, auc))
            print('Recall: {0:.4f}, f1_score: {1:.4f}, precision: {2:.4f}'.format(rec, f1, pre))
            print('='*20)
